# Aluminum Rod Simulation

A program to simulate an isolated rod being heated, and undergoing heat loss via convection and radiation. Considers conduction through the rod.

## Units

- Length: m
- Temperature: K
- Mass: kg
- Energy: J
- Time: s

## Imports

In [1]:
import math
from functools import reduce

import numpy as np

import matplotlib.pyplot as plt

import scipy.optimize as opt

import csv


## Physical Constants

In [2]:
sigma = 5.67 * 10 ** (-8) # Stefan-Boltzmann, W / (m^2 K^4)

## Auxiliary Functions

In [3]:
def celsius_to_kelvin(T_C):
    return T_C + 273.15

## Simulation Parameters + Geometric Properties (customize)

### Sim Run

In [4]:
# rod length
L = 0.3 # m

# dt + N slices -> dx
dt = 0.05 # s

N = 30
dx = L / N # m

r = 0.01 # m

cross_A = math.pi * r ** 2 # m^2

surface_A = np.array([2 * math.pi * r * dx] * N)
surface_A[0] += cross_A
surface_A[N-1] += cross_A

## Ambient Properties (customize)

In [5]:
T_amb = celsius_to_kelvin(20) # K
T_0 = celsius_to_kelvin(20) # initial rod temperature

## Thermal Behaviour

### Initial Temperature Distribution (customize)

In [6]:
def init_temperature():
    return np.array([T_0] * N)

### Power Input (customize)

In [7]:
def power_input(T, P, Pin):
    P[0] += Pin

### Conduction

#### Thermo Function

In [8]:
def conduction(T, P, k):
    
    # conduction temperatures = [T_left T T_right], copy
    # the endpoints have the effect of only conducting within the bar, not to/from the outside
    T_conduct = np.array([T[0]] + np.ndarray.tolist(T) + [T[N-1]])
    
    # P_net = P_in - P_out = kA/L ((T[n-1] - T[n]) - (T[n] - T[n+1])) = kA/L (T[n-1] - 2T[n] + T[n+1])
    P += (k * cross_A / dx) * (T_conduct[0:N] - 2*T_conduct[1:N+1] + T_conduct[2:N+2])
    

### Convection

#### Thermo Function

In [9]:
def convection(T, P, k_c):
    P -= surface_A * k_c * (T - T_amb)

### Radiation

#### Thermo Function

In [10]:
def radiation(T, P, e):
    P -= surface_A * e * sigma * (T ** 4 - T_amb ** 4)

### Calorimetry

#### Parameters (customize)

In [11]:
rho = 2700 # kg/m^3

#### Thermo Function

In [12]:
def calorimetry(T, P, c):
    
    Q = P * dt # P and dt small
    
    dm = rho * cross_A * dx # dx small
    dT = Q / (c * dm)
    
    T += dT

### Master

In [13]:
def run_thermal_iter(T, P, Pin, k, k_c, e, c):
    
    # compute Power
    power_input(T, P, Pin)
    conduction(T, P, k)
    convection(T, P, k_c)
    radiation(T, P, e)
    
    # update Temperature from Power
    calorimetry(T, P, c)
    

## Plotting

### Additional Parameters

In [14]:
#x_vals = np.array(range(N)) * dx

### Plot Functions

In [15]:
def plot_main(x_vals, y_vals):

    fig, ax = plt.subplots()
    
    ax.set_title("Steady State Temperature Distribution in Aluminum Rod")

    ax.set_xlabel("x-coordinate (m)")
    ax.set_ylabel("Tepmerature (K)")

    ax.plot(x_vals, y_vals, 'bx', label='Simulated Data')

    ax.legend()
    
    fig.savefig("foo.png")

## Simulation

### Simulation Auxiliary Functions

In [16]:
def init_power():
    return np.zeros(N)

### Execute Simulation

In [17]:
def run_sim(t_saves, x_points, *thermo_params):
    
    """
    Params:
    t_saves (list[double]) - s, the snapshots in time to capture the temperature distribution data
        in asecending order
    x_points - m, the points of the rod to query the temperature at
    thermo_params (tuple) - tuple of thermo coefficiencts/parameteres
    
    Returns:
    list[double] - a list of temperature distributions, ordered and corresponding to the x_points
        at the specified time snapshots
    """
    
    ## Flow: in every iteration, find net power flow distribution from temperature distribution in previous iteration
    ##       use that to get temperature temperature distribution for the next iteration
    
    t = 0
    tmax = max(t_saves) + 3 * dt
    
    T = init_temperature()
    
    saved_data = []
    
    t_idx = 0
    t_next = t_saves[t_idx]
    
    while True:

        # iteration init
        P = init_power()
        
        # run
        run_thermal_iter(T, P, *thermo_params)
        
        if t >= t_next:
            saved_data.append(probe_data(x_points, T))
            
            t_idx += 1
            if t_idx >= len(t_saves):
                break
            
            t_next = t_saves[t_idx]
        
        t += dt
    
    # Plot temperature distribution
    #plot_main(x_points, T)
    
    return saved_data
    

In [18]:
def position_to_index(x):
    return int(round(x / dx))

def probe_data(x_points, T):
    return list(map(lambda x: T[position_to_index(x)], x_points))
    

## Curve-fit Optimization

### Experimental Setups

In [19]:
def naked_horizontal_rod(t_saves, x_points, Pin, k, k_c, e, c):
    return run_sim(t_saves, x_points, Pin, k, k_c, e, c)

In [20]:
def black_horizontal_rod(t_saves, x_points, Pin, k, k_c, e, c):
    e = 1
    return run_sim(t_saves, x_points, Pin, k, k_c, e, c)

### Actual Data

#### Data files (i.e. testruns)

In [21]:
data_files = ["RunMay17-1.csv", "RunMay17-2.csv"]

#### Probe Points

In [22]:
# test_runs correspond to the test runs in data_files
test_runs = [naked_horizontal_rod,
             black_horizontal_rod]

In [23]:
# x_points should correspond to x-points in data files
x_points = np.array([0.02, 0.085, 0.15, 0.215, 0.28]) # m

In [24]:
t_saves = np.array([0, 600, 1200, 1800]) # s

#### Temperature Sensor Conversion

Currently assuming the temperature-voltage relationship is homogenous for the temperature sensors

In [25]:
def adc_to_temperature(n):
    
    # converts ADC reading to temperature (kelvin)
    
    v = 5 * n / 1024
    
    return celsius_to_kelvin(100.0 * (v - 0.75) + 25.0)

#### Load Data

Note: data files are CSV files, where each row is a measured temperature distribution at a point in time

Row format:
time (ms), ADC readings @ sensors... (to convert to temperature distribution)

The time column (first entry of row) will need to be stripped in data_raw and data

In [26]:
def row_timestamp(row):
    return int(row[0])

def row_data(row):
    return list(map(lambda n: adc_to_temperature(float(n)), row[1:]))

def read_row(row, t, dataset):
    
    # Will add data from row to dataset iff row's timestamp >= t
    # UNITS OVERWRITE: time is in ms
    
    if row_timestamp(row) >= t:
        dataset += row_data(row)
        return True

In [27]:
def load_from_file(filename, t_saves):
    
    # Returns a list of temperature data, corresponding to the t_saves
    # t_saves (s) should be in ascending order
    
    # note that the file data is in ms
    
    file_location = f"data/{filename}"
    
    t_millis = t_saves * 1000;
    
    dataset = []
    
    t_idx = 0
    
    with open(file_location, 'r') as data_file:
        
        csv_reader = csv.reader(data_file)
        
        for row in csv_reader:
            
            if read_row(row, t_millis[t_idx], dataset):
                
                t_idx += 1
                
                if t_idx >= len(t_saves):
                    break
    
    return dataset

In [28]:
def flatten(lst):
    
    lst2 = []
    
    for v in lst:
        lst2 += v
    
    return lst2

In [29]:
# raw_data is a list of testrun_datas
# a testrun_data is a list of time_datas
# a time_data is a list of x-datas (temperature, kelvin)
data_raw = list(map(lambda filename: load_from_file(filename, t_saves), data_files))

In [30]:
# flatten out data_raw as a Numpy array
data = flatten(data_raw)

### Fitting

In [31]:
def squared_residuals(data1, data2):
    return np.linalg.norm(data1 - data2)

In [67]:
def fit_parameters(params):
    
    Pin = params[0]
    k = 210.0 #params[1]
    k_c = 8.0#params[2]
    e = 0.2#params[3]
    
    c = 0.9 * 1000
    
    predicted_raw = list(map(lambda run: run(t_saves, x_points, Pin, k, k_c, e, c), test_runs))
    
    predicted = flatten(flatten(predicted_raw))
    
    return squared_residuals(np.array(predicted), np.array(data))

In [68]:
initial_guess = [6.0]#, 210.0]#, 8.0, 0.5]

In [70]:
import time
timer = - time.time()
opt.minimize(fit_parameters, initial_guess, bounds=[(0.0, 10.0)], method="L-BFGS-B")
timer += time.time()
print("Time taken: " + timer)

KeyboardInterrupt: 